# ITERACIONES: **2.5. Clustering de productos: comparativa de técnicas de análisis clúster.**



**Propósito.**  
Validar la robustez del clustering aplicado al catálogo de productos comparándolo con técnicas alternativas. El objetivo no es sustituir el método actual por defecto, sino **comprobar si K-Means es adecuado** para nuestros datos o si existe una alternativa que aporte **mejor separación/cohesión** y/o **mejor interpretabilidad**.


## Datos de entrada

- **Dataset**: `data/processed/productos_features_norm.csv`  
  (variables numéricas ya estandarizadas: `d_total`, `d_media`, `d_std`, `cv`, `p95`, `mediana`, `precio_medio`, `PC1`, `PC2`, `PC3`; `Product_ID` como identificador).
- Razón de uso: todas las técnicas se benefician de trabajar en la misma escala; garantiza comparabilidad.



## Metodología de comparación

1. **Baseline (Iteración 0): K-Means (k=4)**  
   Partimos de los resultados ya obtenidos (inercia y silhouette vs. *k*, asignación final). Esta iteración sirve de referencia para la comparativa.

2. **Técnicas alternativas**
   - **Iteración 1: Agglomerative Clustering (jerárquico)**  
     Enlace *ward* sobre distancia euclidiana (coherente con datos normalizados). Se evaluarán varios *k*.
   - **Iteración 2: Gaussian Mixture Models (GMM)**  
     Enfoque probabilístico; permite solapamiento entre clusters. Se evaluarán varios *k* y tipos de covarianzas.
   - **Iteración 3: DBSCAN**  
     Clustering basado en densidad; útil para detectar outliers o grupos no esféricos (sin fijar *k*). Se explorarán pares `(eps, min_samples)` razonables.

3. **Métricas de evaluación (internas)**
   - **Silhouette score** (↑ mejor): cohesión intra-cluster y separación inter-cluster.
   - **Davies-Bouldin index** (↓ mejor): ratio de dispersión intra + distancia inter.
   - **Tamaños de cluster**: evitar clusters triviales (muy pequeños) o dominantes excesivos.
   - (Opcional) **Calinski-Harabasz** (↑ mejor): varianza inter/intra.

> Nota: La comparación se centrará en métricas internas (no hay etiquetas “verdaderas”). La **validación de negocio** e **interpretabilidad** se hará en la fase de conclusiones.



## Estructura del cuaderno

- **Iteración 0 — Baseline K-Means**  
  Resumen de resultados (k seleccionado, silhouette, distribución de tamaños).
- **Iteración 1 — Agglomerative**  
  Exploración por *k*: métricas + distribución de tamaños.
- **Iteración 2 — GMM**  
  Exploración por *k* y covarianzas: métricas + distribución de tamaños.
- **Iteración 3 — DBSCAN**  
  Exploración de `(eps, min_samples)`: métricas + tamaños (incluyendo ruido).
- **Tabla comparativa final**  
  Resumen de todas las técnicas/hiperparámetros con métricas lado a lado.
- **Conclusiones**  
  - ¿K-Means es suficiente o hay alternativa superior?  
  - ¿Los resultados son estables y útiles para negocio?  
  - **Decisión**: técnica seleccionada para el proyecto.  
  - Si procede, **siguientes pasos** (crear script definitivo con la técnica elegida).


## Criterios de decisión

1. **Métricas internas**: mejor *silhouette*, menor *Davies-Bouldin* (y, opcionalmente, mayor *Calinski-Harabasz*).  
2. **Estructura razonable**: tamaños de cluster no extremos y sin clusters “vacíos”.  
3. **Parquedad**: preferencia por modelos simples si el rendimiento es similar.  


## Resultados esperados

- Evidencia cuantitativa (gráficas/tablas) que **confirme o refute** la idoneidad de K-Means.  
- Recomendación final y, en su caso, **técnica definitiva** a implementar en script de producción.

> Este cuaderno no genera scripts por iteración. Si se decide cambiar la técnica, se implementará un **único script definitivo** con el algoritmo seleccionado.


## 1.  **Iteración 0.** 
#### ***Baseline K-Means***


In [ ]:

# Script: clustering_productos.py
# ============================================================================

from pathlib import Path
import sys, argparse, logging
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# --------------------------- Raíz del proyecto ------------------------------
def _detect_root_when_no_file():
    here = Path().resolve()
    for p in [here, *here.parents]:
        if (p / "data").is_dir():
            return p
    return here

if "__file__" in globals():
    ROOT_DIR = Path(__file__).resolve().parents[2]
else:
    ROOT_DIR = _detect_root_when_no_file()

DATA_DIR      = ROOT_DIR / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = ROOT_DIR / "reports"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# --------------------------------- Logging ----------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger("clustering_productos")

# --- Parche Jupyter: elimina --f=... del kernel para argparse ---------------
if "ipykernel" in sys.modules or "IPython" in sys.modules:
    sys.argv = [sys.argv[0]]
# ----------------------------------------------------------------------------

# ------------------------------- Utilidades ---------------------------------
NUM_COLS = [
    "d_total", "d_media", "d_std", "cv", "p95", "mediana",
    "precio_medio", "PC1", "PC2", "PC3"
]

def _check_columns(df: pd.DataFrame, cols: list[str]):
    faltan = [c for c in cols if c not in df.columns]
    if faltan:
        raise KeyError(f"Faltan columnas en el dataset de entrada: {faltan}")

def _sample_for_silhouette(X: np.ndarray, max_n: int, random_state: int = 42):
    n = X.shape[0]
    if n <= max_n:
        return X, np.arange(n)
    rng = np.random.default_rng(random_state)
    idx = rng.choice(n, size=max_n, replace=False)
    return X[idx], idx

# ---------------------------------- Core ------------------------------------
def explorar_y_clusterizar(in_path: Path,
                           out_path: Path,
                           k_min: int = 3,
                           k_max: int = 10,
                           force_k: int | None = None,
                           sil_sample: int = 5000,
                           random_state: int = 42,
                           n_init: int = 20):

    # 1) Cargar dataset normalizado
    in_path  = Path(in_path)
    out_path = Path(out_path)
    logger.info(f"Cargando features normalizadas: {in_path}")
    df = pd.read_csv(in_path)

    if "Product_ID" not in df.columns:
        raise KeyError("Se requiere columna 'Product_ID' en el dataset de entrada.")
    _check_columns(df, NUM_COLS)

    X = df[NUM_COLS].astype(float).values
    n, d = X.shape
    logger.info(f"Dimensiones: n={n}, d={d}")

    # 2) Explorar rango de k (si no se fuerza)
    ks = list(range(max(2, k_min), max(k_min, k_max) + 1))
    res_inercia = []
    res_sil = []

    for k in ks:
        logger.info(f"[Exploración] Ajustando KMeans con k={k} ...")
        km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init)
        labels = km.fit_predict(X)
        inertia = float(km.inertia_)
        res_inercia.append({"k": k, "inercia": inertia})

        # Silhouette (requiere k>=2, ya garantizado) — muestreo opcional por eficiencia
        X_sil, idx_sil = _sample_for_silhouette(X, max_n=sil_sample, random_state=random_state)
        lab_sil = labels[idx_sil]
        try:
            sil = float(silhouette_score(X_sil, lab_sil, metric="euclidean"))
        except Exception as e:
            logger.warning(f"Silhouette falló para k={k}: {e}")
            sil = np.nan
        res_sil.append({"k": k, "silhouette": sil})

    df_inercia = pd.DataFrame(res_inercia)
    df_sil = pd.DataFrame(res_sil)

    # Guardar reportes
    path_inercia = REPORTS_DIR / "inercia_vs_k.csv"
    path_sil = REPORTS_DIR / "silhouette_vs_k.csv"
    df_inercia.to_csv(path_inercia, index=False)
    df_sil.to_csv(path_sil, index=False)
    logger.info(f"Guardado: {path_inercia}")
    logger.info(f"Guardado: {path_sil}")

    # 3) Selección de k
    if force_k is not None:
        best_k = int(force_k)
        logger.info(f"Usando k forzado por CLI: k={best_k}")
    else:
        # Elegir k por máximo silhouette (ignorando NaN); si empate, el menor k
        df_sil_valid = df_sil.dropna(subset=["silhouette"])
        if df_sil_valid.empty:
            # fallback: si no hay silhouette válido, usar punto medio del rango
            best_k = int(np.median(ks))
            logger.warning(f"No se pudo calcular silhouette; usando k={best_k} (mediana del rango).")
        else:
            max_sil = df_sil_valid["silhouette"].max()
            candidatos = df_sil_valid.loc[df_sil_valid["silhouette"] == max_sil, "k"].tolist()
            best_k = min(candidatos)
            logger.info(f"Selección automática por silhouette: k={best_k} (silhouette={max_sil:.4f})")

    # 4) Modelo final con best_k
    logger.info(f"Ajustando modelo final KMeans con k={best_k} ...")
    km_final = KMeans(n_clusters=best_k, random_state=random_state, n_init=n_init)
    labels_final = km_final.fit_predict(X)

    # Validación silhouette final (completo o muestreado si es muy grande)
    X_sil_final, idx_sil_final = _sample_for_silhouette(X, max_n=sil_sample, random_state=random_state)
    lab_sil_final = labels_final[idx_sil_final]
    try:
        sil_final = float(silhouette_score(X_sil_final, lab_sil_final, metric="euclidean"))
    except Exception as e:
        logger.warning(f"Silhouette final falló para k={best_k}: {e}")
        sil_final = np.nan

    # 5) Distribución de tamaños de cluster
    _, counts = np.unique(labels_final, return_counts=True)
    dist_sizes = {int(i): int(c) for i, c in enumerate(counts)}
    min_size = counts.min()
    logger.info("=== VALIDACIÓN CLUSTERING ===")
    logger.info(f"k final: {best_k}")
    logger.info(f"Silhouette (final): {sil_final:.4f}" if not np.isnan(sil_final) else "Silhouette (final): NaN")
    logger.info(f"Tamaños de cluster: {dist_sizes} (min={min_size})")

    # 6) Export asignaciones
    df_clusters = df.copy()
    df_clusters["Cluster"] = labels_final
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df_clusters.to_csv(out_path, index=False)
    logger.info(f"Guardado dataset con clusters: {out_path} (filas={len(df_clusters)}, cols={df_clusters.shape[1]})")

    # 7) Devolver info clave
    return {
        "k_final": best_k,
        "silhouette_final": sil_final,
        "sizes": dist_sizes,
        "paths": {
            "clusters": str(out_path),
            "inercia_vs_k": str(path_inercia),
            "silhouette_vs_k": str(path_sil),
        },
    }

# ------------------------------------ CLI -----------------------------------
def parse_args(argv=None):
    p = argparse.ArgumentParser(description="Clustering de productos (K-Means) con exploración de k e informes.")
    p.add_argument("--in",       dest="inp",  type=str, default=str(PROCESSED_DIR / "productos_features_norm.csv"))
    p.add_argument("--out",      dest="outp", type=str, default=str(PROCESSED_DIR / "productos_clusters.csv"))
    p.add_argument("--k-min",    dest="kmin", type=int, default=3)
    p.add_argument("--k-max",    dest="kmax", type=int, default=10)
    p.add_argument("--force-k",  dest="kforce", type=int, default=None, help="Forzar k concreto. Si se indica, salta la selección automática.")
    p.add_argument("--sil-sample", dest="silsample", type=int, default=5000,
                   help="Máximo de observaciones para calcular silhouette (muestreo aleatorio si N>valor).")
    p.add_argument("--seed",     dest="seed", type=int, default=42)
    p.add_argument("--n-init",   dest="ninit", type=int, default=20)

    if argv is None and ("ipykernel" in sys.modules or "IPython" in sys.modules):
        argv = []

    args, _ = p.parse_known_args(argv)
    logger.info("ARGS -> in=%s | out=%s | k=[%d..%d] | force_k=%s | sil_sample=%d | seed=%d | n_init=%d",
                args.inp, args.outp, args.kmin, args.kmax, str(args.kforce), args.silsample, args.seed, args.ninit)
    return args

def main():
    args = parse_args()
    try:
        info = explorar_y_clusterizar(
            in_path=Path(args.inp),
            out_path=Path(args.outp),
            k_min=args.kmin,
            k_max=args.kmax,
            force_k=args.kforce,
            sil_sample=args.silsample,
            random_state=args.seed,
            n_init=args.ninit
        )
        logger.info("Proceso finalizado. k_final=%s | silhouette_final=%s", info["k_final"], info["silhouette_final"])
        logger.info("Rutas: %s", info["paths"])
    except Exception as e:
        logging.exception(f"Error en clustering: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()


2025-08-28 13:41:31 | INFO | ARGS -> in=C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_features_norm.csv | out=C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_clusters.csv | k=[3..10] | force_k=None | sil_sample=5000 | seed=42 | n_init=20
2025-08-28 13:41:31 | INFO | Cargando features normalizadas: C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_features_norm.csv
2025-08-28 13:41:31 | INFO | Dimensiones: n=5938, d=10
2025-08-28 13:41:31 | INFO | [Exploración] Ajustando KMeans con k=3 ...
2025-08-28 13:41:34 | INFO | [Exploración] Ajustando KMeans con k=4 ...
2025-08-28 13:41:34 | INFO | [Exploración] Ajustando KMeans con k=5 ...
2025-08-28 13:41:34 | INFO | [Exploración] Ajustando KMeans con k=6 ...
2025-08-28 13:41:35 | INFO | [Exploración] Ajustando KMeans con k=7 ...
2025-08-28 


## 2.  **Iteración 1.** 
#### ***Agglomerative Clustering***



🔍 **En qué consiste la técnica**
El **clustering aglomerativo** es un método jerárquico que parte de la idea opuesta a K-Means:  
- Cada producto comienza en su propio cluster.  
- En cada paso, se fusionan los dos clusters más similares según una métrica de distancia y un criterio de enlace.  
- El proceso continúa hasta formar el número deseado de clusters (*k*).  

El criterio utilizado en este caso es el **enlace Ward**, que fusiona clusters minimizando la varianza interna. Esto lo hace especialmente adecuado para datos previamente normalizados.


🧩 **Cómo funciona el script**
El script `clustering_productos_agglomerative.py` ejecuta los siguientes pasos:

1. **Entrada**  
   - Lee el dataset `productos_features_norm.csv`.  
   - Utiliza las variables numéricas normalizadas como base para el clustering.

2. **Exploración de *k***  
   - Ajusta modelos de Agglomerative Clustering para distintos valores de *k* en un rango definido (por defecto 3 a 10).  
   - Calcula métricas internas de validación para cada *k*:  
     - **Silhouette score** (↑ mejor).  
     - **Davies–Bouldin index (DBI)** (↓ mejor).  
     - **Calinski–Harabasz (CH)** (↑ mejor).

3. **Selección del número de clusters**  
   - Elige el *k* con mejor **Silhouette score** (o se puede forzar un valor específico desde CLI).  

4. **Entrenamiento final y salida**  
   - Ajusta el modelo final con el *k* seleccionado.  
   - Asigna un cluster a cada producto, añadiendo la columna `Cluster_Agglo`.  
   - Exporta resultados y reportes:  
     - `productos_clusters_agglom.csv` con las asignaciones.  
     - `silhouette_vs_k_agglom.csv`, `davies_bouldin_vs_k_agglom.csv` y `calinski_harabasz_vs_k_agglom.csv` con métricas de exploración.  


🎯 **Objetivo de esta iteración**
Comparar el rendimiento de **Agglomerative Clustering** frente al baseline de K-Means:  
- Ver si obtiene clusters con mejor cohesión/separación (Silhouette ↑, DBI ↓).  
- Evaluar la estabilidad y distribución de tamaños.  
- Comprobar si la estructura jerárquica revela patrones diferentes o más interpretables que K-Means.


In [2]:
# ============================================================================
# Script: clustering_productos_agglomerative.py
# ============================================================================

from pathlib import Path
import sys, argparse, logging
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# --------------------------- Raíz del proyecto ------------------------------
def _detect_root_when_no_file():
    here = Path().resolve()
    for p in [here, *here.parents]:
        if (p / "data").is_dir():
            return p
    return here

if "__file__" in globals():
    ROOT_DIR = Path(__file__).resolve().parents[2]
else:
    ROOT_DIR = _detect_root_when_no_file()

DATA_DIR      = ROOT_DIR / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = ROOT_DIR / "reports"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# --------------------------------- Logging ----------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger("clustering_agglomerative")

# --- Parche Jupyter: elimina --f=... del kernel para argparse ---------------
if "ipykernel" in sys.modules or "IPython" in sys.modules:
    sys.argv = [sys.argv[0]]
# ----------------------------------------------------------------------------

NUM_COLS = [
    "d_total", "d_media", "d_std", "cv", "p95", "mediana",
    "precio_medio", "PC1", "PC2", "PC3"
]

def _check_columns(df: pd.DataFrame, cols: list[str]):
    faltan = [c for c in cols if c not in df.columns]
    if faltan:
        raise KeyError(f"Faltan columnas en el dataset de entrada: {faltan}")

def _fit_predict_agglomerative(X: np.ndarray, k: int, linkage: str, metric: str):
    # Nota: linkage='ward' requiere metric='euclidean' en sklearn.
    if linkage == "ward" and metric != "euclidean":
        metric = "euclidean"
    model = AgglomerativeClustering(
        n_clusters=k, linkage=linkage, metric=metric
    )
    labels = model.fit_predict(X)
    return labels

def explorar_y_clusterizar(in_path: Path,
                           out_path: Path,
                           k_min: int = 3,
                           k_max: int = 10,
                           force_k: int | None = None,
                           linkage: str = "ward",
                           metric: str = "euclidean",
                           sil_sample: int = 7000,
                           random_state: int = 42):
    # 1) Cargar
    in_path  = Path(in_path)
    out_path = Path(out_path)
    logger.info(f"Cargando features normalizadas: {in_path}")
    df = pd.read_csv(in_path)

    if "Product_ID" not in df.columns:
        raise KeyError("Se requiere columna 'Product_ID' en el dataset de entrada.")
    _check_columns(df, NUM_COLS)

    X = df[NUM_COLS].astype(float).values
    n = X.shape[0]
    rng = np.random.default_rng(random_state)

    # 2) Explorar k
    ks = list(range(max(2, k_min), max(k_min, k_max) + 1))
    res_sil, res_dbi, res_ch = [], [], []

    for k in ks:
        logger.info(f"[Exploración] Agglomerative (linkage={linkage}, metric={metric}) con k={k}")
        labels = _fit_predict_agglomerative(X, k, linkage, metric)

        # Silhouette (muestreo si N muy grande)
        if n > sil_sample:
            idx = rng.choice(n, size=sil_sample, replace=False)
            X_s = X[idx]; y_s = labels[idx]
        else:
            X_s, y_s = X, labels

        try:
            sil = float(silhouette_score(X_s, y_s, metric="euclidean"))
        except Exception as e:
            logger.warning(f"Silhouette falló para k={k}: {e}")
            sil = np.nan

        # Davies-Bouldin (↓ mejor) y Calinski-Harabasz (↑ mejor) con todo X (si posible)
        try:
            dbi = float(davies_bouldin_score(X, labels))
        except Exception as e:
            logger.warning(f"Davies-Bouldin falló para k={k}: {e}")
            dbi = np.nan
        try:
            ch = float(calinski_harabasz_score(X, labels))
        except Exception as e:
            logger.warning(f"Calinski-Harabasz falló para k={k}: {e}")
            ch = np.nan

        res_sil.append({"k": k, "silhouette": sil})
        res_dbi.append({"k": k, "davies_bouldin": dbi})
        res_ch.append({"k": k, "calinski_harabasz": ch})

    df_sil = pd.DataFrame(res_sil)
    df_dbi = pd.DataFrame(res_dbi)
    df_ch  = pd.DataFrame(res_ch)

    path_sil = REPORTS_DIR / "silhouette_vs_k_agglom.csv"
    path_dbi = REPORTS_DIR / "davies_bouldin_vs_k_agglom.csv"
    path_ch  = REPORTS_DIR / "calinski_harabasz_vs_k_agglom.csv"
    df_sil.to_csv(path_sil, index=False)
    df_dbi.to_csv(path_dbi, index=False)
    df_ch.to_csv(path_ch, index=False)
    logger.info(f"Guardado: {path_sil}")
    logger.info(f"Guardado: {path_dbi}")
    logger.info(f"Guardado: {path_ch}")

    # 3) Selección de k
    if force_k is not None:
        best_k = int(force_k)
        logger.info(f"Usando k forzado por CLI: k={best_k}")
    else:
        df_sil_valid = df_sil.dropna(subset=["silhouette"])
        if df_sil_valid.empty:
            best_k = int(np.median(ks))
            logger.warning(f"No hay silhouette válido; usando k={best_k} (mediana del rango).")
        else:
            max_sil = df_sil_valid["silhouette"].max()
            candidatos = df_sil_valid.loc[df_sil_valid["silhouette"] == max_sil, "k"].tolist()
            best_k = min(candidatos)
            logger.info(f"Selección automática por silhouette: k={best_k} (silhouette={max_sil:.4f})")

    # 4) Modelo final con best_k
    labels_final = _fit_predict_agglomerative(X, best_k, linkage, metric)

    # Métricas finales
    try:
        sil_final = float(silhouette_score(X if n <= sil_sample else X[rng.choice(n, sil_sample, replace=False)],
                                           labels_final if n <= sil_sample else labels_final[rng.choice(n, sil_sample, replace=False)],
                                           metric="euclidean"))
    except Exception:
        sil_final = np.nan
    try:
        dbi_final = float(davies_bouldin_score(X, labels_final))
    except Exception:
        dbi_final = np.nan
    try:
        ch_final = float(calinski_harabasz_score(X, labels_final))
    except Exception:
        ch_final = np.nan

    # Distribución tamaños
    _, counts = np.unique(labels_final, return_counts=True)
    dist_sizes = {int(i): int(c) for i, c in enumerate(counts)}
    logger.info("=== VALIDACIÓN CLUSTERING (AGGLOMERATIVE) ===")
    logger.info(f"k final: {best_k} | linkage: {linkage} | metric: {metric}")
    logger.info(f"Silhouette final       : {sil_final:.4f}" if not np.isnan(sil_final) else "Silhouette final: NaN")
    logger.info(f"Davies-Bouldin final  : {dbi_final:.4f}" if not np.isnan(dbi_final) else "Davies-Bouldin final: NaN")
    logger.info(f"Calinski-Harabasz final: {ch_final:.2f}" if not np.isnan(ch_final) else "Calinski-Harabasz final: NaN")
    logger.info(f"Tamaños de cluster     : {dist_sizes} (min={counts.min()})")

    # 5) Export asignaciones
    df_out = df.copy()
    df_out["Cluster_Agglo"] = labels_final
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df_out.to_csv(out_path, index=False)
    logger.info(f"Guardado dataset con clusters (agglomerative): {out_path} (filas={len(df_out)})")

    return {
        "k_final": best_k,
        "linkage": linkage,
        "metric": metric,
        "silhouette_final": sil_final,
        "davies_bouldin_final": dbi_final,
        "calinski_harabasz_final": ch_final,
        "sizes": dist_sizes,
        "paths": {
            "clusters": str(out_path),
            "silhouette_vs_k": str(path_sil),
            "davies_bouldin_vs_k": str(path_dbi),
            "calinski_harabasz_vs_k": str(path_ch),
        },
    }

# ------------------------------------ CLI -----------------------------------
def parse_args(argv=None):
    p = argparse.ArgumentParser(description="Clustering aglomerativo con exploración de k y validación interna.")
    p.add_argument("--in",       dest="inp",   type=str, default=str(PROCESSED_DIR / "productos_features_norm.csv"))
    p.add_argument("--out",      dest="outp",  type=str, default=str(PROCESSED_DIR / "productos_clusters_agglom.csv"))
    p.add_argument("--k-min",    dest="kmin",  type=int, default=3)
    p.add_argument("--k-max",    dest="kmax",  type=int, default=10)
    p.add_argument("--force-k",  dest="kforce", type=int, default=None)
    p.add_argument("--linkage",  dest="linkage", type=str, default="ward", choices=["ward","average","complete","single"])
    p.add_argument("--metric",   dest="metric",  type=str, default="euclidean",
                   help="Distancia para enlaces != ward. Con ward se forzará 'euclidean'.")
    p.add_argument("--sil-sample", dest="silsample", type=int, default=7000)
    p.add_argument("--seed",     dest="seed",   type=int, default=42)

    if argv is None and ("ipykernel" in sys.modules or "IPython" in sys.modules):
        argv = []

    args, _ = p.parse_known_args(argv)
    logger.info("ARGS -> in=%s | out=%s | k=[%d..%d] | force_k=%s | linkage=%s | metric=%s | sil_sample=%d",
                args.inp, args.outp, args.kmin, args.kmax, str(args.kforce), args.linkage, args.metric, args.silsample)
    return args

def main():
    args = parse_args()
    try:
        info = explorar_y_clusterizar(
            in_path=Path(args.inp),
            out_path=Path(args.outp),
            k_min=args.kmin,
            k_max=args.kmax,
            force_k=args.kforce,
            linkage=args.linkage,
            metric=args.metric,
            sil_sample=args.silsample,
            random_state=args.seed,
        )
        logger.info("Proceso finalizado. k_final=%s | silhouette=%s | dbi=%s | ch=%s",
                    info["k_final"], info["silhouette_final"], info["davies_bouldin_final"], info["calinski_harabasz_final"])
        logger.info("Rutas: %s", info["paths"])
    except Exception as e:
        logging.exception(f"Error en clustering aglomerativo: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()


2025-08-28 16:02:30 | INFO | ARGS -> in=C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_features_norm.csv | out=C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_clusters_agglom.csv | k=[3..10] | force_k=None | linkage=ward | metric=euclidean | sil_sample=7000
2025-08-28 16:02:30 | INFO | Cargando features normalizadas: C:\Users\crisr\Desktop\Máster Data Science & IA\PROYECTO\PFM2_Asistente_Compras_Inteligente\data\processed\productos_features_norm.csv
2025-08-28 16:02:30 | INFO | [Exploración] Agglomerative (linkage=ward, metric=euclidean) con k=3
2025-08-28 16:02:31 | INFO | [Exploración] Agglomerative (linkage=ward, metric=euclidean) con k=4
2025-08-28 16:02:32 | INFO | [Exploración] Agglomerative (linkage=ward, metric=euclidean) con k=5
2025-08-28 16:02:33 | INFO | [Exploración] Agglomerative (linkage=ward, metric=euclidean) con k=6
2025-08-28 16:0


 📊 **Resumen de métricas**
- **Número de clusters seleccionado (k):** 4  
- **Silhouette final:** 0.3226  
- **Distribución de tamaños:**  
  - Cluster 0 → 210 productos  
  - Cluster 1 → 1233 productos  
  - Cluster 2 → 3394 productos  
  - Cluster 3 → 1101 productos  
  *(mínimo tamaño: 210 productos)*  
- **Dataset resultante:** `productos_clusters.csv` (5.938 productos × 12 columnas)



 ✅ **Conclusiones**
- El modelo jerárquico (Agglomerative con enlace *ward*) ha seleccionado **k=4**, coincidiendo con el resultado obtenido previamente mediante K-Means.  
- El **silhouette (0.3226)** es prácticamente idéntico al de K-Means, lo que indica que **ambas técnicas ofrecen un nivel de cohesión y separación muy similar**.  
- La distribución de productos por cluster es equilibrada, sin clusters triviales o con tamaños insignificantes, lo que confirma la **robustez de la segmentación**.  
- Dado que los resultados son consistentes entre ambos métodos, se puede concluir que la elección de **K-Means como baseline es válida y no sesgada por la técnica**.  
- No obstante, se recomienda continuar la bitácora con otras técnicas (GMM, DBSCAN) para confirmar que no existen estructuras alternativas con mejor separación o clusters residuales que los métodos actuales no detecten.


## 3.  **Iteración 2.** 
#### ***GMM - Gaussian Mixture Models***

## 4.  **Iteración 3.** 
#### ***DBSCAN***